In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.18.0


In [19]:
import tensorflow as tf
import os

import numpy as np

print("TensorFlow version:", tf.__version__)

NUM_FEATURES = 66
NUM_CLASSES = 2

class Model(tf.Module):

  def __init__(self):
    self.model = tf.keras.Sequential([
        tf.keras.layers.Dense(NUM_CLASSES, activation='linear', name='dense_1'),
    ])
    self.model.build(input_shape=[None, NUM_FEATURES])

    self.model.compile(
        optimizer='adam',
        loss=tf.keras.losses.Hinge()
    )

  # The `train` function takes a batch of input instances and labels.
  @tf.function(input_signature=[
      tf.TensorSpec([None, NUM_FEATURES], tf.float32),
      tf.TensorSpec([None, NUM_CLASSES], tf.float32),
  ])
  def train(self, x, y):
    with tf.GradientTape() as tape:
      prediction = self.model(x)
      loss = self.model.loss(y, prediction)
    gradients = tape.gradient(loss, self.model.trainable_variables)
    self.model.optimizer.apply_gradients(
        zip(gradients, self.model.trainable_variables))
    result = {"loss": loss}
    return result

  @tf.function(input_signature=[
      tf.TensorSpec([None, NUM_FEATURES], tf.float32),
  ])
  def infer(self, x):
    logits = self.model(x)
    probabilities = tf.nn.softmax(logits, axis=-1)
    return {
        "output": probabilities,
        "logits": logits
    }

SAVED_MODEL_DIR = "saved_model"
m = Model()

tf.saved_model.save(
    m,
    SAVED_MODEL_DIR,
    signatures={
        'train':
            m.train.get_concrete_function(),
        'test':
            m.infer.get_concrete_function(),
    })

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_DIR)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
]
converter.experimental_enable_resource_variables = True
tflite_model = converter.convert()




with open('custom_model_1_18.tflite', 'wb') as f:
  f.write(tflite_model)

TensorFlow version: 2.18.0
INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets


In [2]:
class MyCustomModel(tf.Module):

  def __init__(self):
    self.model = tf.keras.Sequential([
        tf.keras.layers.Dense(NUM_CLASSES, name='dense')
    ])

    self.model.compile(
        optimizer='adam',
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True))

  # The `train` function takes a batch of input images and labels.
  @tf.function(input_signature=[
      tf.TensorSpec([None, NUM_FEATURES], tf.float32),
      tf.TensorSpec([None, NUM_CLASSES], tf.float32),
  ])
  def train(self, x, y):
    with tf.GradientTape() as tape:
      prediction = self.model(x)
      loss = self.model.loss(y, prediction)
    gradients = tape.gradient(loss, self.model.trainable_variables)
    self.model.optimizer.apply_gradients(
        zip(gradients, self.model.trainable_variables))
    result = {"loss": loss}
    return result

  @tf.function(input_signature=[
      tf.TensorSpec([None, NUM_FEATURES], tf.float32),
  ])
  def infer(self, x):
    logits = self.model(x)
    probabilities = tf.nn.softmax(logits, axis=-1)
    return {
        "output": probabilities,
        "logits": logits
    }

  @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
  def save(self, checkpoint_path):
    tensor_names = [weight.name for weight in self.model.weights]
    tensors_to_save = [weight for weight in self.model.weights]
    tf.raw_ops.Save(
        filename=checkpoint_path, tensor_names=tensor_names,
        data=tensors_to_save, name='save')
    return {
        "checkpoint_path": checkpoint_path
    }

  @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
  def restore(self, checkpoint_path):
    restored_tensors = {}
    for var in self.model.weights:
      restored = tf.raw_ops.Restore(
          file_pattern=checkpoint_path, tensor_name=var.name, dt=var.dtype,
          name='restore')
      var.assign(restored)
      restored_tensors[var.name] = restored
    return restored_tensors
  

SAVED_MODEL_DIR = "saved_model"
model = MyCustomModel()
tf.saved_model.save(
    model,
    SAVED_MODEL_DIR,
    signatures={
        'train': model.train.get_concrete_function(),
        'infer': model.infer.get_concrete_function(),
        'save': model.save.get_concrete_function(),
        'restore': model.restore.get_concrete_function(),
    })
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_DIR)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
]
converter.experimental_enable_resource_variables = True
tflite_model = converter.convert()

NameError: name 'tf' is not defined

NotImplementedError: in user code:

    File "C:\Users\levib\AppData\Local\Temp\ipykernel_2444\285824581.py", line 45, in save  *
        tensors_to_save = [weight.numpy() for weight in self.model.weights]
    File "c:\GUA\gh_repo\GUA\.venv\Lib\site-packages\keras\src\backend\tensorflow\core.py", line 69, in numpy  **
        return self.value.numpy()

    NotImplementedError: numpy() is only available when eager execution is enabled.


In [24]:
import os
import numpy as np
import tensorflow as tf

NUM_FEATURES = 66  # Input size of 66
NUM_CLASSES = 2

class HingeLossModel(tf.keras.Model):
    def __init__(self):
        super(HingeLossModel, self).__init__()
        self.num_features = NUM_FEATURES
        self.num_classes = NUM_CLASSES
        self.dense = tf.keras.layers.Dense(NUM_CLASSES, activation='linear')
        self.optimizer = tf.keras.optimizers.Adam()  # Initialize the optimizer here

    @tf.function(input_signature=[tf.TensorSpec([None, NUM_FEATURES], tf.float32)])
    def infer(self, feature):
        """Performs inference on the given features.

        Args:
          feature: A tensor of input features.

        Returns:
          Map of the softmax output.
        """
        logits = self.dense(feature)
        probabilities = tf.nn.softmax(logits)
        return {"probabilities": probabilities}

    @tf.function(input_signature=[
        tf.TensorSpec([1, NUM_FEATURES], tf.float32),
        tf.TensorSpec([1, NUM_CLASSES], tf.float32),
    ])
    def train(self, feature, label):
        """Trains on a single example."""
        with tf.GradientTape() as tape:
            logits = self.dense(feature)
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(label, logits))
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return {"loss": loss}

    @tf.function(input_signature=[
        tf.TensorSpec([None, NUM_FEATURES], tf.float32),
        tf.TensorSpec([None, NUM_CLASSES], tf.float32),
    ])
    def batch_train(self, feature, label):
        """Trains on a batch of examples."""
        with tf.GradientTape() as tape:
            logits = self.dense(feature)
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(label, logits))
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return {"loss": loss}

    @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
    def save(self, checkpoint_path):
        """Saves the trainable weights to the given checkpoint file.

        Args:
        checkpoint_path: A file path to save the model.
        """
        tensor_names = [self.dense.kernel.name, self.dense.bias.name]  # Get names of the layers' weights and biases
        tensors_to_save = [self.dense.kernel.read_value(), self.dense.bias.read_value()]  # Get the actual tensor values
        tf.raw_ops.Save(
            filename=checkpoint_path,
            tensor_names=tensor_names,
            data=tensors_to_save,
            name='save')
        return {'checkpoint_path': checkpoint_path}


    @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
    def restore(self, checkpoint_path):
        """Restores the serialized trainable weights from the given checkpoint file.

        Args:
        checkpoint_path: A path to a saved checkpoint file.
        """
        restored_tensors = {}

        # Restore the weights and biases of the dense layer
        restored_weights = tf.raw_ops.Restore(
            file_pattern=checkpoint_path,
            tensor_name=self.dense.kernel.name,
            dt=np.float32,
            name='restore')
        self.dense.kernel.assign(restored_weights)  # Assign the restored weights
        restored_tensors['dense/kernel'] = restored_weights

        restored_biases = tf.raw_ops.Restore(
            file_pattern=checkpoint_path,
            tensor_name=self.dense.bias.name,
            dt=np.float32,
            name='restore')
        self.dense.bias.assign(restored_biases)  # Assign the restored biases
        restored_tensors['dense/bias'] = restored_biases

        return restored_tensors

        
    

def convert_and_save(saved_model_dir='saved_model'):
    """Converts and saves the simple dense model with variable number of classes."""
    model = HingeLossModel()

    # Save the model
    tf.saved_model.save(
        model,
        saved_model_dir,
        signatures={
            'train': model.train.get_concrete_function(),
            'infer': model.infer.get_concrete_function(),
        })

    # Convert the model to TensorFlow Lite
    converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
    tflite_model = converter.convert()

    model_file_path = os.path.join('custom_model2.tflite')
    with open(model_file_path, 'wb') as model_file:
        model_file.write(tflite_model)

if __name__ == '__main__':
    convert_and_save()

ValueError: in user code:

    File "C:\Users\levib\AppData\Local\Temp\ipykernel_2444\2163550845.py", line 88, in restore  *
        self.dense.kernel.assign(restored_weights)  # Assign the restored weights
    File "c:\GUA\gh_repo\GUA\.venv\Lib\site-packages\keras\src\backend\common\variables.py", line 225, in assign  **
        if not shape_equal(value.shape, self.shape):
    File "c:\GUA\gh_repo\GUA\.venv\Lib\site-packages\keras\src\backend\common\variables.py", line 549, in shape_equal
        if len(a_shape) != len(b_shape):

    ValueError: Cannot take the length of shape with unknown rank.


In [ ]:
import tensorflow as tf
# Generate a Keras model.
keras_model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(2, input_dim=4, activation='relu', name='x'),
        tf.keras.layers.Dense(1, activation='relu', name='output'),
    ]
)


# Convert the keras model using TFLiteConverter.
# Keras model converter API uses the default signature automatically.
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
tflite_model = converter.convert()

# Print the signatures from the converted model
interpreter = tf.lite.Interpreter(model_content=tflite_model)

signatures = interpreter.get_signature_list()
print(signatures)

INFO:tensorflow:Assets written to: C:\Users\levib\AppData\Local\Temp\tmpdsffet87\assets


c:\GUA\gh_repo\GUA\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
INFO:tensorflow:Assets written to: C:\Users\levib\AppData\Local\Temp\tmpdsffet87\assets


Saved artifact at 'C:\Users\levib\AppData\Local\Temp\tmpdsffet87'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 4), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2348614474576: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2348614475728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2348614478416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2348614479376: TensorSpec(shape=(), dtype=tf.resource, name=None)
{'serving_default': {'inputs': ['keras_tensor'], 'outputs': ['output_0']}}


In [13]:
import os
import numpy as np
import tensorflow as tf

NUM_FEATURES =66  # Input size of 66
NUM_CLASSES = 2 

class SimpleDenseModel(tf.Module):
    """Simple Dense Model Class with variable number of classes."""

    def __init__(self, learning_rate=0.001):
        """Initializes a simple dense model instance with a variable number of classes.

        Args:
          num_classes: Number of output classes for the classification task.
          learning_rate: A learning rate for the optimizer.
        """
        self.num_features = NUM_FEATURES
        self.num_classes = NUM_CLASSES
        # Trainable weights and bias for the dense layer
        self.w = tf.Variable(
            tf.random.uniform((self.num_features, self.num_classes)),
            name='w',
            trainable=True
        )
        self.b = tf.Variable(
            tf.random.uniform((1, self.num_classes)),
            name='b',
            trainable=True
        )

        # Loss function and optimizer
        self.loss_fn = tf.keras.losses.CategoricalCrossentropy()
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    @tf.function(input_signature=[tf.TensorSpec([None, NUM_FEATURES], tf.float32)])
    def infer(self, feature):
        """Performs inference on the given features.

        Args:
          feature: A tensor of input features.

        Returns:
          Map of the softmax output.
        """
        logits = tf.matmul(feature, self.w) + self.b
        return {'output': tf.nn.softmax(logits)}

    @tf.function(input_signature=[
        tf.TensorSpec([1, NUM_FEATURES], tf.float32),
        tf.TensorSpec([1, NUM_CLASSES], tf.float32), 
    ])
    def train(self, feature, label):
        """Runs one training step with the given features and labels.

        Args:
          feature: A tensor of input features.
          label: A tensor of class labels (size [batch_size, num_classes]).

        Returns:
          Map of the training loss.
        """
        with tf.GradientTape() as tape:
            logits = tf.matmul(feature, self.w) + self.b
            prediction = tf.nn.softmax(logits)
            loss = self.loss_fn(label, prediction)
        gradients = tape.gradient(loss, [self.w, self.b])
        self.optimizer.apply_gradients(zip(gradients, [self.w, self.b]))
        result = {'loss': loss}
        for grad in gradients:
            result[grad.name] = grad
        return result

    @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
    def save(self, checkpoint_path):
        """Saves the trainable weights to the given checkpoint file.

        Args:
          checkpoint_path: A file path to save the model.
        """
        tensor_names = [self.w.name, self.b.name]
        tensors_to_save = [self.w.read_value(), self.b.read_value()]
        tf.raw_ops.Save(
            filename=checkpoint_path,
            tensor_names=tensor_names,
            data=tensors_to_save,
            name='save')
        return {'checkpoint_path': checkpoint_path}

    @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
    def restore(self, checkpoint_path):
        """Restores the serialized trainable weights from the given checkpoint file.

        Args:
          checkpoint_path: A path to a saved checkpoint file.
        """
        restored_tensors = {}
        restored = tf.raw_ops.Restore(
            file_pattern=checkpoint_path,
            tensor_name=self.w.name,
            dt=np.float32,
            name='restore')
        self.w.assign(restored)
        restored_tensors['w'] = restored
        restored = tf.raw_ops.Restore(
            file_pattern=checkpoint_path,
            tensor_name=self.b.name,
            dt=np.float32,
            name='restore')
        self.b.assign(restored)
        restored_tensors['b'] = restored
        return restored_tensors

def convert_and_save(saved_model_dir='saved_model'):
    """Converts and saves the simple dense model with variable number of classes."""
    model = SimpleDenseModel()

    tf.saved_model.save(
        model,
        saved_model_dir,
        signatures={
            'train': model.train.get_concrete_function(),
            'infer': model.infer.get_concrete_function(),
            'save': model.save.get_concrete_function(),
            'restore': model.restore.get_concrete_function(),
        })

    # Convert the model
    converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
    converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,  # enable TensorFlow Lite ops.
        tf.lite.OpsSet.SELECT_TF_OPS  # enable TensorFlow ops.
    ]
    converter.experimental_enable_resource_variables = True
    tflite_model = converter.convert()

    model_file_path = os.path.join('fixed_model.tflite')
    with open(model_file_path, 'wb') as model_file:
        model_file.write(tflite_model)


if __name__ == '__main__':
    # Example usage with 5 classes
    convert_and_save()


INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets


In [ ]:
NUM_FEATURES = 66
NUM_CLASSES = 2

class Model(tf.Module):
  
  def __init__(self):
    self.model = tf.keras.Sequential([
        tf.keras.layers.Dense(NUM_CLASSES, activation='relu', name='dense_1'),
    ])
    self.model.build(input_shape=[None, NUM_FEATURES])

    self.model.compile(
        optimizer='adam',
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True))

  # The `train` function takes a batch of input instances and labels.

  @tf.function(input_signature=[tf.TensorSpec(N))



  @tf.function(input_signature=[
      tf.TensorSpec([None, NUM_FEATURES], tf.float32),
      tf.TensorSpec([None, NUM_CLASSES], tf.float32),
  ])
  def train(self, x, y):
    with tf.GradientTape() as tape:
      prediction = self.model(x)
      loss = self.model.loss(y, prediction)
    gradients = tape.gradient(loss, self.model.trainable_variables)
    self.model.optimizer.apply_gradients(
        zip(gradients, self.model.trainable_variables))
    result = {"loss": loss}
    return result

  @tf.function(input_signature=[
      tf.TensorSpec([None, NUM_FEATURES], tf.float32),
  ])
  def infer(self, x):
    logits = self.model(x)
    probabilities = tf.nn.softmax(logits, axis=-1)
    return {
        "output": probabilities,
        "logits": logits
    }

  @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
  def save(self, checkpoint_path):
    tensor_names = [weight.name for weight in self.model.weights]
    tensors_to_save = [weight.read_value() for weight in self.model.weights]
    tf.raw_ops.Save(
        filename=checkpoint_path, tensor_names=tensor_names,
        data=tensors_to_save, name='save')
    return {
        "checkpoint_path": checkpoint_path
    }

  @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
  def restore(self, checkpoint_path):
    restored_tensors = {}
    for var in self.model.weights:
      restored = tf.raw_ops.Restore(
          file_pattern=checkpoint_path, tensor_name=var.name, dt=var.dtype,
          name='restore')
      var.assign(restored)
      restored_tensors[var.name] = restored
    return restored_tensors

c:\GUA\gh_repo\GUA\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


AttributeError: 'function' object has no attribute 'get_concrete_function'